In [1]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
from datasets import load_dataset
from utils_emotiontraining import *

Device set to use mps


In [2]:
model = TFAutoModel.from_pretrained("distilroberta-base")
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")
emotion_dataset = load_dataset("jellyshroom/go_emotions_augmented")


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFRobertaModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


README.md:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.00M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/344k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/348k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/46536 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5427 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5426 [00:00<?, ? examples/s]

In [6]:
small_train_dataset = emotion_dataset['train'].select(range(20000))
small_test_dataset = emotion_dataset['test'].select(range(1000))
small_validation_dataset = emotion_dataset['validation'].select(range(1000))

small_emotion_dataset = {
    'train': small_train_dataset,
    'test': small_test_dataset,
    'validation': small_validation_dataset
}

In [7]:
emotions_id2label = {
    0: 'admiration',
    1: 'amusement',
    2: 'anger',
    3: 'annoyance',
    4: 'approval',
    5: 'caring',
    6: 'confusion',
    7: 'curiosity',
    8: 'desire',
    9: 'disappointment',
    10: 'disapproval',
    11: 'disgust',
    12: 'embarrassment',
    13: 'excitement',
    14: 'fear',
    15: 'gratitude',
    16: 'grief',
    17: 'joy',
    18: 'love',
    19: 'nervousness',
    20: 'optimism',
    21: 'pride',
    22: 'realization',
    23: 'relief',
    24: 'remorse',
    25: 'sadness',
    26: 'surprise',
    27: 'neutral'
}

emotions_label2id = {v: k for k, v in emotions_id2label.items()}

In [8]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True, return_tensors='tf')

# Tokenize the small dataset
small_emotions_encoded = {}
small_emotions_encoded['train'] = small_emotion_dataset['train'].map(tokenize, batched=True, batch_size=None)
small_emotions_encoded['test'] = small_emotion_dataset['test'].map(tokenize, batched=True, batch_size=None)
small_emotions_encoded['validation'] = small_emotion_dataset['validation'].map(tokenize, batched=True, batch_size=None)

print(small_emotions_encoded['train'])

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'labels', 'id', 'input_ids', 'attention_mask'],
    num_rows: 20000
})


In [9]:
import numpy as np
import tensorflow as tf
from collections import Counter

NUM_CLASSES = 28  # Define the number of classes (matches emotions_id2label)

def create_multi_hot_labels(example):
    """Convert the list of labels into a multi-hot encoded vector."""
    multi_hot_label = np.zeros(NUM_CLASSES, dtype=np.float32)
    if 'labels' in example and isinstance(example['labels'], list) and len(example['labels']) > 0:
        for label_id in example['labels']:
            if isinstance(label_id, int) and 0 <= label_id < NUM_CLASSES:
                multi_hot_label[label_id] = 1.0
    example['multi_hot_labels'] = multi_hot_label
    return example

# Apply the conversion to add the new multi-hot label column
small_emotions_encoded['train'] = small_emotions_encoded['train'].map(create_multi_hot_labels)
small_emotions_encoded['test'] = small_emotions_encoded['test'].map(create_multi_hot_labels)
small_emotions_encoded['validation'] = small_emotions_encoded['validation'].map(create_multi_hot_labels)

# Remove the original 'labels' column and any leftover 'label_int'
columns_to_remove = [col for col in ['label_int', 'labels'] if col in small_emotions_encoded['train'].features]
if columns_to_remove:
    print(f"Removing existing columns: {columns_to_remove}")
    small_emotions_encoded['train'] = small_emotions_encoded['train'].remove_columns(columns_to_remove)
    small_emotions_encoded['test'] = small_emotions_encoded['test'].remove_columns(columns_to_remove)
    if 'validation' in small_emotions_encoded:
        small_emotions_encoded['validation'] = small_emotions_encoded['validation'].remove_columns(columns_to_remove) # Added for validation set

# Rename the new column 'multi_hot_labels' to 'labels'
if 'multi_hot_labels' in small_emotions_encoded['train'].features:
    print("Renaming 'multi_hot_labels' to 'labels'")
    small_emotions_encoded['train'] = small_emotions_encoded['train'].rename_column('multi_hot_labels', 'labels')
if 'multi_hot_labels' in small_emotions_encoded['test'].features:
    small_emotions_encoded['test'] = small_emotions_encoded['test'].rename_column('multi_hot_labels', 'labels')
if 'validation' in small_emotions_encoded and 'multi_hot_labels' in small_emotions_encoded['validation'].features:
    small_emotions_encoded['validation'] = small_emotions_encoded['validation'].rename_column('multi_hot_labels', 'labels') # Added for validation set


Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Removing existing columns: ['labels']
Renaming 'multi_hot_labels' to 'labels'


In [10]:
print("Calculating sample weights...")
# Get all multi-hot labels from the training set as a NumPy array
train_labels_np = np.array(small_emotions_encoded['train']['labels'])
# Count frequency of each label (column-wise sum)
label_counts = np.sum(train_labels_np, axis=0)
total_samples = len(train_labels_np)

# Calculate weight for each class (inverse frequency, smoothed)
class_weights_calc = {}
for i in range(NUM_CLASSES):
    count = label_counts[i] if label_counts[i] > 0 else 1
    class_weights_calc[i] = total_samples / (NUM_CLASSES * count)

# Calculate weight for each sample: max weight of its positive labels
sample_weights_np = np.zeros(total_samples, dtype=np.float32)
for i in range(total_samples):
    sample_label_indices = np.where(train_labels_np[i] == 1.0)[0]
    if len(sample_label_indices) > 0:
        sample_weights_np[i] = max(class_weights_calc[idx] for idx in sample_label_indices)
    else:
        # Assign a default weight (e.g., 1.0 or average) for samples with no positive labels
        sample_weights_np[i] = 1.0
print("Sample weights calculated.")

Calculating sample weights...
Sample weights calculated.


In [11]:
# Set format to tensorflow
feature_cols = ["input_ids", "attention_mask"]
label_col = "labels"
cols_to_set_format = feature_cols + [label_col]

actual_train_cols = list(small_emotions_encoded['train'].features)
actual_test_cols = list(small_emotions_encoded['test'].features)
actual_validation_cols = list(small_emotions_encoded['validation'].features)
final_train_cols = [col for col in cols_to_set_format if col in actual_train_cols]
final_test_cols = [col for col in cols_to_set_format if col in actual_test_cols]
final_validation_cols = [col for col in cols_to_set_format if col in actual_validation_cols]

if all(col in final_train_cols for col in cols_to_set_format) and \
   all(col in final_test_cols for col in cols_to_set_format) and \
   all(col in final_validation_cols for col in cols_to_set_format):
    print("Setting dataset format to TensorFlow")
    # Don't set format yet, extract numpy arrays first, then create dataset
else:
     raise ValueError(f"Error: Could not find all necessary columns. Train has: {actual_train_cols}, Test has: {actual_test_cols}. Needed: {cols_to_set_format}")


# Extract features and labels as numpy arrays before creating dataset
train_features_np = {col: np.array(small_emotions_encoded['train'][col]) for col in feature_cols}
train_labels_np = np.array(small_emotions_encoded['train']['labels']) # Already have this from weight calc

test_features_np = {col: np.array(small_emotions_encoded['test'][col]) for col in feature_cols}
test_labels_np = np.array(small_emotions_encoded['test']['labels'])

validation_features_np = {col: np.array(small_emotions_encoded['validation'][col]) for col in feature_cols}
validation_labels_np = np.array(small_emotions_encoded['validation']['labels'])


Setting dataset format to TensorFlow


In [12]:

# Create TensorFlow datasets
BATCH_SIZE = 32 # Keep batch size reasonable

print("Creating tf.data.Dataset objects with sample weights...")
# Modify train_dataset to yield (features, labels, sample_weights)
train_dataset = tf.data.Dataset.from_tensor_slices(
    (train_features_np, train_labels_np, sample_weights_np)
)
train_dataset = train_dataset.shuffle(len(sample_weights_np)).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE) # Add prefetch

# Test dataset remains (features, labels)
test_dataset = tf.data.Dataset.from_tensor_slices(
    (test_features_np, test_labels_np)
)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE) # Add prefetch

validation_dataset = tf.data.Dataset.from_tensor_slices(
    (validation_features_np, validation_labels_np)
)
validation_dataset = validation_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE) # Add prefetch


print("Datasets created successfully.")


Creating tf.data.Dataset objects with sample weights...
Datasets created successfully.


In [19]:
class BERTForClassification(tf.keras.Model):
    def __init__(self, bert_model, num_classes):
        super().__init__()
        self.bert = bert_model
        self.num_classes = num_classes # STORE num_classes
        # Change activation to 'sigmoid' for multi-label classification
        self.fc = tf.keras.layers.Dense(num_classes, activation='sigmoid')

    def call(self, inputs):
        # Make sure we handle the case when inputs is a dictionary
        outputs = self.bert(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            return_dict=True
        )
        pooled_output = outputs.pooler_output
        return self.fc(pooled_output)

    # ADD THIS METHOD
    def get_config(self):
        config = super().get_config()
        config.update({
            "num_classes": self.num_classes
            # bert_model is not included as it's complex and loaded separately
        })
        return config

In [20]:
# Define NUM_CLASSES if not defined globally earlier
try:
    NUM_CLASSES
except NameError:
    NUM_CLASSES = 28 # Set default if run out of order

# Create a shared emotion prediction function for multi-label output
def predict_emotion(text, model, threshold=0.5):
    """Predict multiple emotions for a given text using the provided model and threshold"""
    inputs = tokenizer(text, return_tensors='tf', padding=True, truncation=True)
    predictions = model(inputs) # Shape: (1, NUM_CLASSES)

    # --- Add this line temporarily ---
    print(f"Raw probabilities for '{text}': {predictions[0].numpy()}")
    # --- End of added line ---

    predicted_labels_indices = tf.where(predictions[0] > threshold).numpy().flatten()
    predicted_emotions = []
    confidences = []
    if len(predicted_labels_indices) > 0:
        for index in predicted_labels_indices:
            predicted_emotions.append(emotions_id2label[index])
            confidences.append(float(predictions[0][index]))
    else:
        # Optional: If no label passes threshold, predict the highest one or 'neutral'
        highest_prob_index = tf.argmax(predictions, axis=1).numpy()[0]
        predicted_emotions.append(emotions_id2label[highest_prob_index])
        confidences.append(float(predictions[0][highest_prob_index]))


    return {
        'text': text,
        'emotions': predicted_emotions,
        'confidences': confidences
    }

# Define test texts to use for both untrained and trained models
test_texts = [
    "I'm so happy today!",
    "This makes me really angry.",
    "I'm feeling very sad and disappointed.",
    "That's really interesting, tell me more.",
    "I am both excited and nervous about the presentation.", # Example with multiple emotions
]

In [21]:
# Create an untrained model for baseline comparison
# Ensure the base model 'model' is loaded correctly from cell 2
untrained_classifier = BERTForClassification(model, num_classes=NUM_CLASSES)

# Compile the model for multi-label classification
untrained_classifier.compile(
    optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=2e-5),
    # Use BinaryCrossentropy for multi-label with sigmoid activation
    loss=tf.keras.losses.BinaryCrossentropy(),
    # Use BinaryAccuracy for multi-label evaluation
    metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy')]
)

print("Predictions with UNTRAINED model (random weights - multi-label):")
print("-------------------------------------------------------------")

for text in test_texts:
    result = predict_emotion(text, untrained_classifier, threshold=0.5)
    print(f"Text: {result['text']}")
    print(f"Predicted emotions: {result['emotions']}")
    # Zip confidences with emotions for clarity
    emotion_confidence_pairs = list(zip(result['emotions'], result['confidences']))
    print(f"Confidences: {emotion_confidence_pairs}")
    print()

Predictions with UNTRAINED model (random weights - multi-label):
-------------------------------------------------------------
Raw probabilities for 'I'm so happy today!': [0.3647775  0.46604145 0.71257216 0.6748999  0.5231001  0.4923539
 0.81130844 0.22087218 0.33193052 0.32434133 0.2523739  0.36111632
 0.5680941  0.57176626 0.3669782  0.4474013  0.27805743 0.5727703
 0.90865403 0.24848849 0.74004817 0.5476425  0.59768933 0.4063118
 0.29717106 0.529662   0.4651278  0.23796488]
Text: I'm so happy today!
Predicted emotions: ['anger', 'annoyance', 'approval', 'confusion', 'embarrassment', 'excitement', 'joy', 'love', 'optimism', 'pride', 'realization', 'sadness']
Confidences: [('anger', 0.7125721573829651), ('annoyance', 0.6748998761177063), ('approval', 0.5231000781059265), ('confusion', 0.8113084435462952), ('embarrassment', 0.5680940747261047), ('excitement', 0.5717662572860718), ('joy', 0.5727702975273132), ('love', 0.9086540341377258), ('optimism', 0.7400481700897217), ('pride', 0.5

In [22]:

try:
    NUM_CLASSES
except NameError:
    NUM_CLASSES = 28

# Define the model - ensure 'model' (the base BERT model) is loaded
classifier = BERTForClassification(model, num_classes=NUM_CLASSES)

# Compile the model for multi-label classification with more metrics
print("Compiling model with AUC, Precision, Recall...")
classifier.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=2e-5), # Consider trying AdamW later
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.AUC(multi_label=True, name='auc'),
        tf.keras.metrics.Precision(name='precision'), # How many selected items are relevant?
        tf.keras.metrics.Recall(name='recall') # How many relevant items are selected?
        ]
)
print("Model compiled.")

Compiling model with AUC, Precision, Recall...
Model compiled.


In [23]:

print("Starting multi-label model training with sample weights...")
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=3, mode='max', restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint('best_emotion_model.keras', save_best_only=True, monitor='val_auc', mode='max')
]

history = classifier.fit(
    train_dataset,
    epochs=5,
    validation_data=validation_dataset,
    callbacks=callbacks # Uncomment to use callbacks
)
print("Training finished.")

# Evaluate the model on the test set
print("Evaluating model on test set...")
results = classifier.evaluate(test_dataset, verbose=1) # Use verbose=1 to see progress

# Print evaluation results dynamically based on compiled metrics
print("\nTest Set Evaluation Results:")
for name, value in zip(classifier.metrics_names, results):
    print(f"- {name}: {value:.4f}")

# test_loss = results[classifier.metrics_names.index('loss')]
# test_auc = results[classifier.metrics_names.index('auc')]
# print(f"\nTest Loss: {test_loss:.4f}")
# print(f"Test AUC: {test_auc:.4f}")

Starting multi-label model training with sample weights...
Epoch 1/5
625/625 [==============================] - 3146s 5s/step - loss: 0.1398 - accuracy: 0.9568 - auc: 0.8232 - precision: 0.4528 - recall: 0.1283 - val_loss: 0.1168 - val_accuracy: 0.9632 - val_auc: 0.8921 - val_precision: 0.6888 - val_recall: 0.2659
Epoch 2/5
625/625 [==============================] - 3169s 5s/step - loss: 0.0976 - accuracy: 0.9636 - auc: 0.9131 - precision: 0.6691 - recall: 0.2684 - val_loss: 0.1080 - val_accuracy: 0.9622 - val_auc: 0.8998 - val_precision: 0.6287 - val_recall: 0.3016
Epoch 3/5
625/625 [==============================] - 3159s 5s/step - loss: 0.0844 - accuracy: 0.9646 - auc: 0.9334 - precision: 0.6525 - recall: 0.3392 - val_loss: 0.1055 - val_accuracy: 0.9620 - val_auc: 0.9078 - val_precision: 0.6059 - val_recall: 0.3389
Epoch 4/5
625/625 [==============================] - 3036s 5s/step - loss: 0.0754 - accuracy: 0.9657 - auc: 0.9463 - precision: 0.6532 - recall: 0.3946 - val_loss: 0.1052

In [25]:
print("Predictions with TRAINED multi-label model:")
print("----------------------------------------")

# Use the updated shared function with the trained model
prediction_threshold = 0.2 # Adjust threshold as needed based on validation performance

test_texts = TEST_TEXTS

print("Loading best model weights from checkpoint...")
classifier.load_weights('best_emotion_model.keras')

for text in test_texts:
    result = predict_emotion(text, classifier, threshold=prediction_threshold)
    print(f"Text: {result['text']}")
    print(f"Predicted emotions: {result['emotions']}")
    # Zip confidences with emotions for clarity
    emotion_confidence_pairs = list(zip(result['emotions'], result['confidences']))
    print(f"Confidences: {emotion_confidence_pairs}")
    # print(f"Confidences: {[f'{c:.4f}' for c in result['confidences']]}")
    print()

Predictions with TRAINED multi-label model:
----------------------------------------
Loading best model weights from checkpoint...
Raw probabilities for 'I'm so happy today!': [3.5804652e-02 6.7219036e-03 3.5327272e-03 4.2225751e-03 2.2938062e-02
 2.1542782e-02 1.8839212e-03 4.4516129e-03 4.6389336e-03 2.3737701e-03
 9.3714846e-04 1.1744318e-03 3.6886306e-03 8.9674823e-02 2.7164354e-03
 2.1942558e-02 8.9404261e-04 9.5895094e-01 3.8458385e-02 2.9750334e-03
 8.6716171e-03 3.1830974e-02 1.1759102e-02 7.2076959e-03 3.7947532e-03
 1.0251633e-02 2.5655765e-03 6.0006138e-03]
Text: I'm so happy today!
Predicted emotions: ['joy']
Confidences: [('joy', 0.958950936794281)]

Raw probabilities for 'This makes me really angry.': [0.0045552  0.01158904 0.9307607  0.18818492 0.00901619 0.00386939
 0.00210112 0.00174721 0.00395207 0.01632867 0.0182901  0.07202452
 0.0048206  0.00815627 0.0109376  0.01113673 0.0043633  0.00679844
 0.00830816 0.00545055 0.00529313 0.02591576 0.00787402 0.01033634
 0.0160

In [24]:
from transformers import pipeline

# Load the pre-trained model and tokenizer
model = 'borisn70/bert-43-multilabel-emotion-detection'
tokenizer = 'borisn70/bert-43-multilabel-emotion-detection'

# Create a pipeline for sentiment analysis
nlp = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

testTexts = TEST_TEXTS
result = nlp(testTexts)

# Print the result
print(result)


Device set to use mps:0


[{'label': 'joy', 'score': 0.9998683929443359}, {'label': 'anger', 'score': 0.9998956918716431}, {'label': 'sadness', 'score': 0.9999432563781738}, {'label': 'neutral', 'score': 0.7713387608528137}, {'label': 'fear', 'score': 0.9935625791549683}, {'label': 'fear', 'score': 0.9996892213821411}, {'label': 'anger', 'score': 0.9703493714332581}, {'label': 'joy', 'score': 0.9454078674316406}, {'label': 'joy', 'score': 0.6393844485282898}, {'label': 'neutral', 'score': 0.7897074222564697}, {'label': 'sadness', 'score': 0.9980188608169556}, {'label': 'fear', 'score': 0.5129643082618713}, {'label': 'neutral', 'score': 0.9996825456619263}, {'label': 'joy', 'score': 0.9929666519165039}, {'label': 'fear', 'score': 0.9995587468147278}]
